In [83]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [66]:
import utils
data = utils.load_data([
    "../results/synthetic/1000/set1/"
])

In [73]:
groups = {
    "num_endogenous": data.num_endogenous.unique(), 
    "markovianity": data.markovianity.unique() 
}

In [ ]:
bins=5
by = "quant_ps1"
#by = "disc_ps1"

In [84]:
# discretize
data["disc_ps1"] = pd.cut(data.ps1, bins=bins)
data["quant_ps1"] = pd.qcut(data.ps1, q=bins, duplicates="drop")

# get exact rows
exact = data[data.method == "exact"]

# get only ps1=1 where selctor is true
ps1_1 = data[(data.method == "EMCC") & (data.ps1 >= 1.0) & (data.selector)]

# add excat column to all rows
data2 = data.merge(exact[["file", "pns_l", "pns_u"]], on="file")
data2 = data2[(data2.method=="EMCC") & (data2.selector)]

data2["r_pns_l"] = (data2.pns_l_x - data2.pns_l_y)
data2["r_pns_u"] = (data2.pns_u_x - data2.pns_u_y)

for key in groups:
    for value in groups[key]:
        fig, ax = plt.subplots(2,1, figsize=(15,15))
        
        filter = data2[key] == value

        axu = data2[filter].boxplot("r_pns_u", by=by,  ax=ax[0])
        axl = data2[filter].boxplot("r_pns_l", by=by,  ax=ax[1])

        axu.set_title("difference with exact upper")
        axl.set_title("difference with exact lower")

        axu.set_ylim(-0.25,1)
        axl.set_ylim(-0.25,1)

        fig.suptitle(f"Errors on Pns by discretized Ps1. {key} == {value}")
        
        fig.savefig(f"plots/error_by_{bins}_{by}_{key}_{value}.jpg")
        plt.close()
        
